In [119]:
reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [120]:

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from lxml import etree
import requests

In [121]:
headers = {
    'x-devtools-emulate-network-conditions-client-id': "c60c3ba6-26e8-45ce-9f9f-0b5521437cf6",
    'upgrade-insecure-requests': "1",
    'user-agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36",
    'accept': "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    'referer': "http://isa.epfl.ch/imoniteur_ISAP/%21gedpublicreports.htm?ww_i_reportmodel=133685247",
    'accept-encoding': "gzip, deflate, sdch",
    'accept-language': "es-ES,es;q=0.8",
    'cookie': "LANGUE_LOGIN=fr",
    'cache-control': "no-cache"
}

data = requests.get("http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247", headers=headers)

soup = BeautifulSoup(data.content, 'html.parser')
queries = soup.find_all('select')
queries_map = {}
for s in queries:
    queries_map[s.get('name')] = {}
    map = queries_map[s.get('name')]
    options = s.find_all('option')
    for o in options:
        if not o.string:
            value = ""
        else:
            value = o.string
        map[value] = o.get('value')

print(queries_map)

{'ww_x_PERIODE_PEDAGO': {'': 'null', 'Stage automne 3ème année': '953137', 'Bachelor semestre 6b': '2226785', 'Master semestre 3': '2230128', 'Bachelor semestre 5': '942120', 'Mise à niveau': '2063602308', 'Semestre printemps': '2754553', 'Stage automne 4ème année': '2226616', 'Projet Master automne': '249127', 'Master semestre 4': '2230140', 'Bachelor semestre 1': '249108', 'Mineur semestre 1': '2335667', 'Mineur semestre 2': '2335676', 'Master semestre 2': '942192', 'Bachelor semestre 3': '942155', 'Master semestre 1': '2230106', 'Bachelor semestre 2': '249114', 'Projet Master printemps': '3781783', 'Stage printemps 3ème année': '983606', 'Bachelor semestre 5b': '2226768', 'Stage printemps 4ème année': '2226626', 'Stage printemps master': '2227132', 'Bachelor semestre 6': '942175', 'Semestre automne': '953159', 'Bachelor semestre 4': '942163'}, 'ww_x_HIVERETE': {'': 'null', "Semestre d'automne": '2936286', 'Semestre de printemps': '2936295'}, 'ww_x_PERIODE_ACAD': {'': 'null', '2014-2

In [122]:
def getData(unite_acad, periode_acad, periode_pedago, semester_type):
    unite_acad_id = queries_map['ww_x_UNITE_ACAD'][unite_acad]
    periode_acad_id = queries_map['ww_x_PERIODE_ACAD'][periode_acad]
    periode_pedago_id = queries_map['ww_x_PERIODE_PEDAGO'][periode_pedago]
    semester_type_id = queries_map['ww_x_HIVERETE'][semester_type]
    url = "http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter"

    querystring = {"ww_b_list":"1",
                   "ww_i_reportmodel":"133685247",
                   "ww_c_langue":"",
                   "ww_i_reportModelXsl":"133685270",
                   "zz_x_UNITE_ACAD": unite_acad,
                   "ww_x_UNITE_ACAD": unite_acad_id,
                   "zz_x_PERIODE_ACAD": periode_acad,
                   "ww_x_PERIODE_ACAD":periode_acad_id,
                   "zz_x_PERIODE_PEDAGO":periode_pedago,
                   "ww_x_PERIODE_PEDAGO":periode_pedago_id,
                   "zz_x_HIVERETE":semester_type,
                   "ww_x_HIVERETE":semester_type_id,
                   "dummy":"ok"}

    headers = {
        'upgrade-insecure-requests': "1",
        'user-agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36",
        'accept': "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        'referer': "http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247",
        'accept-encoding': "gzip, deflate, sdch",
        'accept-language': "es-ES,es;q=0.8",
        'cookie': "LANGUE_LOGIN=fr",
        'cache-control': "no-cache",
        'postman-token': "65d96955-59bb-daa2-5136-807135f5b89a"
        }

    text = BeautifulSoup(requests.request("GET", url, headers=headers, params=querystring).text, 'html.parser')
    gps = list(text.find_all('table'))[2].find_all('tr')[1].td.a.get('onclick').split('ww_x_GPS=')[1].split("')")[0]
    payload = {'ww_x_GPS': int(gps),
               'ww_i_reportModel':133685247, 
               'ww_i_reportModelXsl': 133685271,
               'ww_x_UNITE_ACAD': unite_acad_id,
               'ww_x_PERIODE_ACAD': periode_acad_id,
               'ww_x_PERIODE_PEDAGO': periode_pedago_id,
               'ww_x_HIVERETE': semester_type_id
              }
    r = requests.get("http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.XLS", params=payload)
    return r.text

In [123]:
df_final = None
df_year = None

def addData(unite_acad, periode_acad, periode_pedago, semester_type, list_semesters):
    unite_acad_id = queries_map['ww_x_UNITE_ACAD'][unite_acad]
    periode_acad_id = queries_map['ww_x_PERIODE_ACAD'][periode_acad]
    periode_pedago_id = queries_map['ww_x_PERIODE_PEDAGO'][periode_pedago]
    semester_type_id = queries_map['ww_x_HIVERETE'][semester_type]

    
    soup = BeautifulSoup(getData(unite_acad,periode_acad,periode_pedago,semester_type), 'html.parser')
    rows = soup.find_all('tr')
    header_cols = rows[2].find_all('th')
    col_titles, data = [], {}
    for c in header_cols:
        data[c.string] = []
        col_titles.append(c.string)
    [col_titles.append(semestre[9:]) for semestre in list_semesters]
    for semestre in list_semesters:
        data[semestre[9:]] = [ None ] * ( len(rows) - 3)
    df = pd.DataFrame(columns=col_titles)
    global df_final, df_year
    if df_final is None:
        df_final = df.copy()
    if df_year is None:
        df_year = df.copy()
    
    
    for h in range(3, len(rows)):
        values = rows[h].find_all('td')
        for v in range(len(values)-2):
            data[header_cols[v].string].append(values[v].string)
        data[periode_pedago[9:]] = int(periode_acad[:-5])
    df = pd.DataFrame.from_dict(data)
    return df



In [135]:
unite_acad = "Informatique"
periode_acad = list(queries_map['ww_x_PERIODE_ACAD'].keys())
periode_acad.remove('')
periode_pedago = []
for p in list(queries_map['ww_x_PERIODE_PEDAGO'].keys()):
    if 'Bachelor' in p and "b" not in p:
        periode_pedago.append(p)
periode_pedago = sorted(periode_pedago)
df_final = None
for year in sorted(periode_acad):
    print(year,)
    df_year = None
    for i in periode_pedago:
        a = addData("Informatique", year, i, "", periode_pedago)
        df_year = df_year.append(a, ignore_index = True)
    df_final = df_final.combine_first(df_year)
df_final

2007-2008
Bachelor semestre 1  
90   
Bachelor semestre 2  
72   
Bachelor semestre 3  
50   
Bachelor semestre 4  
48   
Bachelor semestre 5  
48   
Bachelor semestre 6  
38   

2008-2009
Bachelor semestre 1  
96   
Bachelor semestre 2  
74   
Bachelor semestre 3  
53   
Bachelor semestre 4  
51   
Bachelor semestre 5  
57   
Bachelor semestre 6  
51   

2009-2010
Bachelor semestre 1  
117   
Bachelor semestre 2  
101   
Bachelor semestre 3  
54   
Bachelor semestre 4  
52   
Bachelor semestre 5  
75   
Bachelor semestre 6  
60   

2010-2011
Bachelor semestre 1  
153   
Bachelor semestre 2  
125   
Bachelor semestre 3  
51   
Bachelor semestre 4  
51   
Bachelor semestre 5  
62   
Bachelor semestre 6  
52   

2011-2012
Bachelor semestre 1  
166   
Bachelor semestre 2  
125   
Bachelor semestre 3  
77   
Bachelor semestre 4  
75   
Bachelor semestre 5  
63   
Bachelor semestre 6  
52   

2012-2013
Bachelor semestre 1  
198   
Bachelor semestre 2  
170   
Bachelor semestre 3  
75   
Bac

,Civilité,Ecole Echange,Filičre opt.,Mineur,No Sciper,Nom Prénom,Orientation Bachelor,Orientation Master,Spécialisation,Statut,Type Echange,semestre 1,semestre 2,semestre 3,semestre 4,semestre 5,semestre 6
0,Monsieur,None,None,NaN,169569,Arévalo Christian,NaN,NaN,NaN,Présent,None,2007.0,NaN,NaN,NaN,NaN,NaN
1,Monsieur,None,None,NaN,174905,Aubelle Flavien,NaN,NaN,NaN,Présent,None,2007.0,NaN,NaN,NaN,NaN,NaN
2,Monsieur,None,None,NaN,173922,Badoud Morgan,NaN,NaN,NaN,Présent,None,2007.0,NaN,NaN,NaN,NaN,NaN
3,Monsieur,None,None,NaN,179406,Baeriswyl Jonathan,NaN,NaN,NaN,Présent,None,2007.0,NaN,NaN,NaN,NaN,NaN
4,Monsieur,None,None,NaN,179428,Barroco Michael,NaN,NaN,NaN,Présent,None,2007.0,NaN,NaN,NaN,NaN,NaN
5,Monsieur,None,None,NaN,179324,Belfis Nicolas,NaN,NaN,NaN,Présent,None,2007.0,NaN,NaN,NaN,NaN,NaN
6,Monsieur,None,None,NaN,174597,Beliaev Stanislav,NaN,NaN,NaN,Présent,None,2007.0,NaN,NaN,NaN,NaN,NaN
7,Monsieur,None,None,NaN,179449,Bindschaedler Vincent,NaN,NaN,NaN,Présent,None,2007.0,NaN,NaN,NaN,NaN,NaN
8,Monsieur,None,None,NaN,178553,Bloch Marc-Olivier,NaN,NaN,NaN,Présent,None,2007.0,NaN,NaN,NaN,NaN,NaN
9,Monsieur,None,None,NaN,179426,Bloch Remi,NaN,NaN,NaN,Présent,None,2007.0,NaN,NaN,NaN,NaN,NaN


In [140]:
df_final[df_final['No Sciper'] == '169569']

,Civilité,Ecole Echange,Filičre opt.,Mineur,No Sciper,Nom Prénom,Orientation Bachelor,Orientation Master,Spécialisation,Statut,Type Echange,semestre 1,semestre 2,semestre 3,semestre 4,semestre 5,semestre 6
0,Monsieur,None,None,NaN,169569,Arévalo Christian,NaN,NaN,NaN,Présent,None,2007.0,NaN,NaN,NaN,NaN,NaN
90,Monsieur,None,None,NaN,169569,Arévalo Christian,NaN,NaN,NaN,Présent,None,2008.0,2007.0,NaN,NaN,NaN,NaN
399,Monsieur,None,6 - Visual computing,NaN,169569,Arévalo Christian,NaN,NaN,NaN,Présent,None,NaN,2014.0,2012.0,2011.0,2010.0,2009.0
